library import

In [7]:
import time
import random
from colorsys import hsv_to_rgb
import board
from digitalio import DigitalInOut, Direction
from PIL import Image, ImageDraw, ImageFont
from adafruit_rgb_display import st7789
import numpy as np
from collections import deque

Define Joystick

In [8]:
class Joystick:
    def __init__(self):
        self.cs_pin = DigitalInOut(board.CE0)
        self.dc_pin = DigitalInOut(board.D25)
        self.reset_pin = DigitalInOut(board.D24)
        self.BAUDRATE = 24000000

        self.spi = board.SPI()
        self.disp = st7789.ST7789(
                    self.spi,
                    height=240,
                    y_offset=80,
                    rotation=180,
                    cs=self.cs_pin,
                    dc=self.dc_pin,
                    rst=self.reset_pin,
                    baudrate=self.BAUDRATE,
                    )

        # Input pins:
        self.button_A = DigitalInOut(board.D5)
        self.button_A.direction = Direction.INPUT

        self.button_B = DigitalInOut(board.D6)
        self.button_B.direction = Direction.INPUT

        self.button_L = DigitalInOut(board.D27)
        self.button_L.direction = Direction.INPUT

        self.button_R = DigitalInOut(board.D23)
        self.button_R.direction = Direction.INPUT

        self.button_U = DigitalInOut(board.D17)
        self.button_U.direction = Direction.INPUT

        self.button_D = DigitalInOut(board.D22)
        self.button_D.direction = Direction.INPUT

        self.button_C = DigitalInOut(board.D4)
        self.button_C.direction = Direction.INPUT

        # Turn on the Backlight
        self.backlight = DigitalInOut(board.D26)
        self.backlight.switch_to_output()
        self.backlight.value = True

        # Create blank image for drawing.
        # Make sure to create image with mode 'RGB' for color.
        self.width = self.disp.width
        self.height = self.disp.height

Define Ball

In [9]:
class Ball:
    # 공모양, x축 속도, y축 가속도, 
    def __init__(self, width, height, size = 8):
        self.appearance = 'circle'
        self.state = None
        self.items_queue = deque()
        self.position = np.array([width/2 - size, height/2 - size, width/2 + size, height/2 + size])
        self.speed = 5
        self.velocity = 0
        self.acceleration = 1
        self.outline = "#000000"

    # 움직임 정의
    def move(self, command = None):

        # y축에 대한 정의
        self.velocity += self.acceleration
        self.position[3] += self.velocity
        self.position[1] += self.velocity

        if(self.position[3] >= 240 and self.velocity > 0):
            self.velocity = -11

        # shoot 아이템 때문에 들어간 코드
        if( self.speed > 5 ):
            self.speed -= self.acceleration

        # 아이템 사용할 때 사용될 코드(조건)
        if self.items_queue and not joystick.button_B.value:
            current_item = self.items_queue[0]
            current_item.use(self)
            self.items_queue.popleft()

        # 가만히 있을 때
        if command['move'] == False:
            self.state = None
        
        # 움직일 때
        else:
            self.state = 'move'
                
            if command['left_pressed']:
                self.position[0] -= self.speed
                self.position[2] -= self.speed
                
            if command['right_pressed']:
                self.position[0] += self.speed
                self.position[2] += self.speed

    # 아이템을 먹는 거 체크
    def collision_check(self, items):
        for item in items:
            collision = self.overlap(self.position, item.position)

            if collision:
                self.items_queue.append(item)
                items.remove(item)

    def overlap(self, ego_position, other_position):
        return ego_position[0] < other_position[2] and ego_position[1] < other_position[3] \
            and ego_position[2] > other_position[0] and ego_position[3] > other_position[1]


Define Item

In [10]:
class Item:
    def __init__(self, spawn_position, size = 13, image_path=None):
        self.appearance = 'image'
        self.position = np.array([spawn_position[0] - size, spawn_position[1] - size, spawn_position[0] + size, spawn_position[1] + size])
        self.outline = "#00FF00"
        self.is_collected = False
        self.image = None

        # 만약 image_path가 존재한다면 그걸 사용해라
        if image_path:
            self.image = Image.open(image_path)

    def use(self, ball):
        self.is_collected = True

# 점프아이템에 대한 정의
class Jump(Item):
    def __init__(self, spawn_position):
        super().__init__(spawn_position, image_path="Jump.png")

    #높게 뛰게
    def use(self, ball):
        super().use(ball)
        ball.velocity = -10

# 슛아이템에 대한 정의
class Shoot(Item):
    def __init__(self, spawn_position):
        super().__init__(spawn_position, image_path="Shoot.png")

    # 빠르게 날아가기
    def use(self, ball):
        super().use(ball)
        ball.speed = 16
        ball.velocity -= 3


Make joystick, Image, Draw Object

In [11]:
joystick = Joystick()
my_image = Image.new("RGB", (joystick.width, joystick.height)) #도화지!
my_draw = ImageDraw.Draw(my_image) #그리는 도구!

In [19]:
my_circle = Ball(joystick.width, joystick.height)
my_draw.rectangle((0, 0, joystick.width, joystick.height), fill=(255, 255, 255, 100))

# 게임 세계에 아이템 추가
items = [Jump((200, joystick.height -14)), Shoot((20, joystick.height - 14))]  # 필요한 만큼 추가


while True:
    command = {'move': False, 'left_pressed': False, 'right_pressed': False}

    if not joystick.button_L.value:  # 왼쪽 버튼이 눌린 경우
        command['left_pressed'] = True
        command['move'] = True

    if not joystick.button_R.value:  # 오른쪽 버튼이 눌린 경우
        command['right_pressed'] = True
        command['move'] = True

    my_circle.move(command)
    my_circle.collision_check(items)  # 아이템과의 충돌 확인 및 아이템 사용

    # 그리기 로직 (순서가 중요함)
    my_draw.rectangle((0, 0, joystick.width, joystick.height), fill=(255, 255, 255, 100))

    # 아이템을 가지고 있다면 주황색으로, 아니면 노란색으로
    if( my_circle.items_queue ):
        my_draw.ellipse(tuple(my_circle.position), outline=my_circle.outline, fill=(255, 165, 0, 255))
    else:
        my_draw.ellipse(tuple(my_circle.position), outline=my_circle.outline, fill=(255, 255, 0))

    # 아이템 그리기
    for jump_item in items:
        if not jump_item.is_collected:
            # 아이템 이미지를 그리기
            if jump_item.image:
                my_image.paste(jump_item.image, (int(jump_item.position[0]), int(jump_item.position[1])))
            else:
                my_draw.rectangle(tuple(jump_item.position), outline=jump_item.outline, fill=None)


    # Update display
    joystick.disp.image(my_image)

KeyboardInterrupt: 